In [3]:
import pandas as pd

In [4]:
# Dataset loading

# Reading csv files
# gddAprOct = pd.read_csv("dataset/10_4231_R72F7KK2/Agro-Climatic Data by County/gddAprOct.csv")
# gridInfo = pd.read_csv("dataset/10_4231_R72F7KK2/Agro-Climatic Data by County/gridInfo.csv")
# yielddata = pd.read_csv("dataset/10_4231_R72F7KK2/Agro-Climatic Data by County/yielddata.csv")
crime_data = pd.read_csv("../../../../Downloads/Crimes_-_2001_to_present.csv")

# Reading txt files
# txt_data = ''
# with open('dataset/CAF_Sensor_Dataset_2/CAF_CropCodes.txt','r') as file:
#     txt_data = file.read()

In [ ]:
# Earlier analysis

# Dataset-1
gddAprOct.head()
gddAprOct['year'].head()
gddAprOct['year'].min()
gddAprOct['year'].max()
gddAprOct.shape
gddAprOct['gddm60'].isna().sum()
gddAprOct.max().max()
len(gddAprOct.loc[gddAprOct['gddm5'] == 0.0])

# Dataset-2
gridInfo.head()
gridInfo.shape

# Dataset-3
yielddata.head()
yielddata['corn'].isna().sum()
yielddata.shape

In [3]:
from pygooglechart import PieChart3D

def python_pie3D(data, labels) :
  # initialize chart object, 250 x 250 pixels
  chart = PieChart3D(250, 250)

  # pass your data to the chart object
  chart.add_data(data)

  # make labels for the slices
  chart.set_pie_labels(labels)

  # render the image
  chart.download('revenue_east_europe.png')

In [5]:
import seaborn as sns 

# Analysis harness
analysis_df = crime_data

# To show different metrics
# analysis_df.info()
# analysis_df['Arrest'].describe()

# import numpy as np
# from matplotlib import cm
# color = cm.inferno_r(np.linspace(.4,.8, 30))

# analysis_df.groupby(['Primary Type', 'Arrest']).size().unstack().plot(kind='barh', stacked=True, figsize=(10,10))
# df2 = analysis_df.groupby(['Primary Type', 'Arrest'])['Primary Type'].count().unstack('Arrest').fillna(0)
# print (df2.head())
# df2[[False,True]].plot(kind='bar', stacked=True)

# analysis_df['Primary Type'].value_counts().plot(kind='barh', figsize=(10,10), color=color)
# analysis_df.isnull().sum().plot(kind='barh', figsize=(10,10), color=color)
# analysis_df[analysis_df['Arrest']==False]['Primary Type'].value_counts().plot(kind='pie', figsize=(10,10))
# df = analysis_df[analysis_df['Arrest']==False]['Primary Type'].value_counts()
# python_pie3D(df.data, df.index)

# To get null values
# analysis_df.isnull().sum()

# Finding co-relation
# correlation_matrix = analysis_df.corr().round(2)
# sns.heatmap(data=correlation_matrix, annot=True)

In [4]:
# converts txt to csv files
import csv 

def txt_to_csv(txt_file):
    csv_file = txt_file.split('/')[-1].split('.')[0] + '.csv'
    csv_file = '/'.join(txt_file.split('/')[:-1]) + '/' + csv_file
    
    inp_data = list()
    with open(txt_file, 'r') as in_file:
        stripped = (line.strip() for line in in_file)
        inp_data = [line.split("\t") for line in stripped if line]
    
    # creating dataframe
    df = pd.DataFrame(inp_data[1:-2], columns=inp_data[0])
    print(df.head())
    df.to_csv(csv_file)

In [ ]:
txt_to_csv('dataset/CAF_Sensor_Dataset_2/CAF_CropID.txt')

In [5]:
import requests

class API_client:
    def __init__(self, url, username, password):
        self._url = url
        self._auth = (username, password)
        
    def set_url(self, url):
        '''
        to reset url
        '''
        self._url = url
        
    def get_response(self):
        '''
        get response from url
        '''
        resp = requests.get(self._url, auth=self._auth)
        
        if resp.status_code != 200:
            # This means something went wrong.
            raise ApiError('GET rest API issue: {}'.format(resp.status_code))
            
        return resp.json()
    
    def put_response(self, data):
        '''
        put response
        '''
        resp = requests.post(self._url, auth=self._auth, json=data)
        
        if resp.status_code != 201:
            raise ApiError('POST rest API issue: {}'.format(resp.status_code))
            
        return 'POST successful'
        
    def __str__(self):
        return 'This is a REST API functions wrapper'

In [ ]:
# analysis_df = analysis_df.to_json(orient='index')
host_name = 'http://localhost:8091'
rest_client = API_client(couchbase_url, 'administrator', 'nainyjain')

# Exploration
url_1 = '/pools/default/buckets'

rest_client.set_url(host_name+url_1)
print (rest_client.get_response())

In [13]:
import json
from couchbase.cluster import Cluster
from couchbase.cluster import PasswordAuthenticator
cluster = Cluster('couchbase://localhost')
authenticator = PasswordAuthenticator('administrator', 'nainyjain')
cluster.authenticate(authenticator)
cb = cluster.open_bucket('sample_data')

for index, doc in enumerate(analysis_df[:100000].T.to_dict().values()):
    doc_id = 'k' + str(index)
    cb.upsert(doc_id, doc)

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(data, f)